In [ ]:
import os
import torch
import torch.nn as nn
import torch.distributed.rpc as rpc
import torch.multiprocessing as mp
import torch.distributed as dist
import torch.nn.functional as F
from torch import autograd
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import mnist
import time

import denpendencies

In [ ]:
os.environ['MASTER_ADDR'] = '127.0.0.1'
os.environ['MASTER_PORT'] = '29503'
rank = 1
world_size = 2
dist.init_process_group(backend='gloo', rank=rank, world_size=2)

initialize and connect

In [ ]:
class RemotePassBegin(autograd.Function):
    @staticmethod
    def forward(ctx, input):
        dist.send(torch.tensor(0), 0)
        dist.send(input, 0)
        return input

    @staticmethod
    def backward(ctx, grad_outputs):
        x = torch.rand([128, 1152])
        
        dist.recv(x, 0)
        return x


class RemotePassEnd(autograd.Function):
    @staticmethod
    def forward(ctx, input):
        # ctx.input = input
        x = torch.rand([128, 10])
        dist.recv(x, 0)
        return x

    @staticmethod
    def backward(ctx, grad_output):
        dist.send(torch.tensor(1), 0)
        dist.send(grad_output, 0)
        return None

build network

In [ ]:
func = nn.CrossEntropyLoss()
net = NetClient()


train_batch_size = 128
test_batch_size = 64
learning_rate = 0.01
momentum = 0.5
num_epoches = 10
# create two datasets
transforms = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])
train_dataset = mnist.MNIST(
    './data', train=True, transform=transforms, download=True)
test_dataset = mnist.MNIST('./data', train=False, transform=transforms)
# iterate
train_loader = DataLoader(
    train_dataset, batch_size=train_batch_size, shuffle=True)
test_loader = DataLoader(
    test_dataset, batch_size=test_batch_size, shuffle=False)


optimizer = optim.SGD(net.parameters(),
                      lr=learning_rate, momentum=momentum)

optimizer, datasets and net

In [ ]:
def train():
    for epoch in range(num_epoches):
        train_loss = 0
        train_acc = 0
        time_start = time.time()
        i = 0
        for img, label in train_loader:
            i = i +1
            if i == 468:
                break
            img = img
            label = label
            x = net(img)
            x = RemotePassBegin.apply(x)
            x = RemotePassEnd.apply(x)
            criterion = nn.CrossEntropyLoss()
            loss = criterion(x, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # calculate loss
            train_loss += loss.item()
            # calculate accuracy
            pred = x.argmax(dim=1)
            train_acc += (pred == label).sum().item() / img.size(0)
            #print(i)
        train_loss = train_loss / len(train_loader)
        train_acc = train_acc / len(train_loader)
        train_losses.append(train_loss)
        train_accs.append(train_accs)
        # 日志输出
        time_end = time.time()
        print('time cost', time_end-time_start, 's')
        print("Epoch: {}, Train loss: {:.4f}, Train acc: {:.4f}".format(
            epoch, train_loss, train_acc))

define training functions